In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T

pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))


In [ ]:

nInst = 50
currentPos = np.zeros(nInst)

# Initialize models and scalers
models = [Sequential([
    LSTM(50, activation='relu', input_shape=(20, 1)),
    Dense(1)
]) for _ in range(nInst)]

for model in models:
    model.compile(optimizer='adam', loss='mse')

scalers = [StandardScaler() for _ in range(nInst)]

def prepare_data(data, lookback=20):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i + lookback)])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

def getMyPosition(prcSoFar):
    global currentPos, models, scalers
    (nins, nt) = prcSoFar.shape
    
    if nt < 50:  # We need at least 50 days of data for meaningful predictions
        return np.zeros(nins)
    
    predictions = []
    
    for i in range(nins):
        prices = prcSoFar[i, :]
        scaled_prices = scalers[i].fit_transform(prices.reshape(-1, 1))
        
        X, y = prepare_data(scaled_prices)
        
        # Train the model
        models[i].fit(X, y, epochs=5, verbose=0)
        
        # Make a prediction
        last_20_scaled = scaled_prices[-20:].reshape(1, 20, 1)
        predicted_scaled = models[i].predict(last_20_scaled)
        predicted_price = scalers[i].inverse_transform(predicted_scaled)[0, 0]
        
        # Calculate the predicted return
        current_price = prices[-1]
        predicted_return = (predicted_price - current_price) / current_price
        predictions.append(predicted_return)
    
    # Normalize predictions
    predictions = np.array(predictions)
    predictions = predictions / np.sum(np.abs(predictions))
    
    # Calculate target positions
    target_position = np.array([int(x) for x in dlrPosLimit * predictions / prcSoFar[:, -1]])
    
    # Implement a conservative position change
    position_change = 0.2 * (target_position - currentPos)
    
    # Update current position
    currentPos = np.array([int(x) for x in currentPos + position_change])
    
    # Ensure we're within position limits
    current_prices = prcSoFar[:, -1]
    posLimits = np.array([int(x) for x in dlrPosLimit / current_prices])
    currentPos = np.clip(currentPos, -posLimits, posLimits)
    
    return currentPos

# The rest of the code (calcPL function and the main execution) remains the same

In [ ]:
def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(500, 751):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        print(newPos)
        if (totDVolume > 0):
            ret = value / totDVolume
        print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
              (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)


(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd


In [ ]:
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)